# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
#Read Csv File In
data = pd.read_csv("city_weather_data.csv")

#Check to see data read in
data.head()

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind,Country,Date
0,Hithadhoo,-0.6000,73.0833,82.85,82.85,75,100,20.67,MV,1635632139
1,Hilo,19.7297,-155.0900,79.70,82.26,70,40,8.05,US,1635632140
2,Husavik,66.0449,-17.3389,39.11,39.11,89,100,17.78,IS,1635632140
3,Mataura,-46.1927,168.8643,49.66,49.66,96,100,12.59,NZ,1635632141
4,Laguna,38.4210,-121.4238,68.76,74.23,71,20,4.00,US,1635632141


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
humidity = data["Humidity"]
locations = data[["Latitude","Longitude"]]

In [48]:
#Api key set up
gmaps.configure(api_key = g_key)

#Set how map initially displays
maps = gmaps.figure(center=(0,0), zoom_level=1, layout={'width': '550px','height': '600px'})

#Establish Heat Layer Settings
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity = 300, point_radius = 5, dissipating =False)

#Apply to map
maps.add_layer(heat_layer)

#Display map with heat layers
maps

Figure(layout=FigureLayout(height='600px', width='550px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
#Narrow down the cities to fit weather conditions
city_param=data.loc[(data["Temperature"]<85) & (data["Temperature"]>75) & (data["Wind"]<5) & (data["Cloudiness"]<5)]

#Drop any rows will null values.
city_param=city_param.dropna(how='any')

#Reset index 
city_param.reset_index(inplace=True)

#Display dataframe
city_param

,index,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind,Country,Date
0,65,Isangel,-19.5500,169.2667,82.51,82.51,61,0,3.44,VU,1635632163
1,169,Saint George,37.1041,-113.5841,76.21,79.63,30,1,4.00,US,1635632012
2,300,Dzaoudzi,-12.7887,45.2699,75.09,75.09,83,0,3.44,YT,1635632252
3,331,San Patricio,28.0170,-97.5169,78.42,86.41,18,0,2.42,US,1635632265
4,334,Abu Dhabi,24.4667,54.3667,82.42,82.42,74,0,3.44,AE,1635632266
5,455,Salalah,17.0151,54.0924,78.89,78.89,61,0,3.44,OM,1635632312
6,466,Los Lunas,34.8062,-106.7334,75.36,79.95,21,0,1.01,US,1635632103


### Hotel Map
* Store into variable named `hotel`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [50]:
#Create Dataframes
hotel = city_param.loc[:,["City","Country","Latitude","Longitude"]]

# Establish Column to be filled
hotel["Hotel Name"] = ""

#Check Data Frame
hotel.head()

,City,Country,Latitude,Longitude,Hotel Name
0,Isangel,VU,-19.5500,169.2667,
1,Saint George,US,37.1041,-113.5841,
2,Dzaoudzi,YT,-12.7887,45.2699,
3,San Patricio,US,28.0170,-97.5169,
4,Abu Dhabi,AE,24.4667,54.3667,


In [51]:
#Establish url for api pull
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [52]:
#Set parameters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [53]:
#Loop through data and identify hotels for cities in hotel data frame print zero element for each city 
for index, x in hotel.iterrows():
    latitude = x["Latitude"]
    longitude = x["Longitude"]
    c_name = x["City"]
    
    params["location"] = f"{latitude},{longitude}"

    print(f"Index Number {index}: {c_name}.")
    response = requests.get(url, params=params).json()
    
    results = response['results']
   
    try:
        if len(results) > 0 : 
            
            if "name" in  results[0]:
                print(f"Hotel in {c_name} is {results[0]['name']}.")
                hotel_name = results[0]['name']
                hotel.loc[index, "Hotel Name"] = hotel_name
            else : 
                hotel.loc[index, "Hotel Name"] = "N/A"
        else :
            hotel.loc[index, "Hotel Name"] = "N/A"
                
    except (KeyError, IndexError):
        print("None")
       
print("End")

Index Number 0: Isangel.
Hotel in Isangel is Quarumanu area.
Index Number 1: Saint George.
Hotel in Saint George is Red Lion Hotel & Conference Center St. George.
Index Number 2: Dzaoudzi.
Hotel in Dzaoudzi is 2.0 Hotel.
Index Number 3: San Patricio.
Hotel in San Patricio is Americas Best Value Inn Sinton.
Index Number 4: Abu Dhabi.
Hotel in Abu Dhabi is Emirates Palace.
Index Number 5: Salalah.
Hotel in Salalah is Salalah Gardens Hotel.
Index Number 6: Los Lunas.
Hotel in Los Lunas is Days Inn by Wyndham Los Lunas.
End


In [54]:
#Add to dataframe andRemove the data not identified 
hotel = hotel.dropna(how='any')
#Check data frame
hotel.head(7)

,City,Country,Latitude,Longitude,Hotel Name
0,Isangel,VU,-19.5500,169.2667,Quarumanu area
1,Saint George,US,37.1041,-113.5841,Red Lion Hotel & Conference Center St. George
2,Dzaoudzi,YT,-12.7887,45.2699,2.0 Hotel
3,San Patricio,US,28.0170,-97.5169,Americas Best Value Inn Sinton
4,Abu Dhabi,AE,24.4667,54.3667,Emirates Palace
5,Salalah,OM,17.0151,54.0924,Salalah Gardens Hotel
6,Los Lunas,US,34.8062,-106.7334,Days Inn by Wyndham Los Lunas


In [55]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel.iterrows()]
locations = hotel[["Latitude", "Longitude"]]

In [56]:
# Add marker layer ontop of heat map
indicator = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Display figure
maps.add_layer(indicator)
maps

Figure(layout=FigureLayout(height='600px', width='550px'))